### Create the tables for the Flowline Services - located in the reference schema / EGIS database 

In [ ]:
from helper_functions.shared_functions import execute_sql, move_data_from_viz_to_egis

#### SQL statement that creates the CONUS Channels and Thresholds reference table for the Flowlines Service

In [ ]:
## NOTE: some columns had the datatype cast to match those in the current reference table so that the mapx would not need to be adjusted
sql = f'''
DROP TABLE IF EXISTS dev.flowlines_conus;
SELECT channels.feature_id, 
    channels.strm_order::BIGINT, 
	channels.name::TEXT, 
	channels.huc6, 
	CAST('3.0' AS double precision) as nwm_vers, 
	channels.geom, 
	thresholds.high_water_threshold, 
	thresholds.rf_2_0_17c, 
	thresholds.rf_5_0_17c, 
	thresholds.rf_10_0_17c, 
	thresholds.rf_25_0_17c, 
	thresholds.rf_50_0_17c, 
	row_number() over() as oid,
	channels.feature_id::TEXT AS feature_id_str
INTO dev.flowlines_conus
FROM derived.channels_conus AS channels
JOIN derived.recurrence_flows_conus AS thresholds ON channels.feature_id = thresholds.feature_id
'''
execute_sql(sql)

#### Move the CONUS dev table from viz to an "on deck" version in the egis database

In [ ]:
move_data_from_viz_to_egis('dev.flowlines_conus', 'reference.channels_conus_ondeck')

#### Create a "retired" CONUS reference table and rename the "on deck" table to the live version

In [ ]:
execute_sql('''
    DROP TABLE IF EXISTS reference.channels_conus_retired;
    ALTER TABLE reference.channels_conus RENAME TO channels_conus_retired;
    ALTER TABLE reference.channels_conus_ondeck RENAME TO channels_conus;
''', 'egis')

#### SQL statement that creates the ALASKA Channels and Thresholds reference table for the Flowlines Service

In [ ]:
sql = f'''
DROP TABLE IF EXISTS dev.flowlines_ak;
SELECT channels.feature_id, 
    channels.strm_order, 
	channels.name::TEXT, 
	channels.huc6, 
	CAST('3.0' AS double precision) as nwm_vers, 
	channels.geom, 
	thresholds.high_water_threshold, 
	thresholds.rf_2_0_17c, 
	thresholds.rf_5_0_17c, 
	thresholds.rf_10_0_17c, 
	thresholds.rf_25_0_17c, 
	thresholds.rf_50_0_17c, 
	row_number() over() as oid,
	channels.feature_id::TEXT AS feature_id_str
INTO dev.flowlines_ak
FROM derived.channels_alaska AS channels
JOIN derived.recurrence_flows_ak AS thresholds ON channels.feature_id = thresholds.feature_id
'''
execute_sql(sql)

#### Move the Alaska dev table from viz to "on deck" version in the egis database

In [ ]:
move_data_from_viz_to_egis('dev.flowlines_ak', 'reference.channels_ak_ondeck')

#### Create a "retired" Alaska reference table and rename the "on deck" table to the live version

In [ ]:
execute_sql('''
    DROP TABLE IF EXISTS reference.channels_ak_retired;
    ALTER TABLE reference.channels_ak RENAME TO channels_ak_retired;
    ALTER TABLE reference.channels_ak_ondeck RENAME TO channels_ak;
''', 'egis')